<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_threads_python_M1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python**

In [ ]:
%%writefile tp2.py
import sys
import os
import random
import threading
MIN_MATRIX_SIZE = 5
MAX_MATRIX_SIZE = 20
MIN_MATRIX_CELL_VALUE = -32
MAX_MATRIX_CELL_VALUE = 32

def validateArgs(args):
    if not len(args) == 2:
        print("Cantidad incorrecta de argumentos")
        os._exit(1)
    size = int(args[1])
    if size > MAX_MATRIX_SIZE or size < MIN_MATRIX_SIZE:
        print("El tamaño debe estar entre {min} y {max}".format(min=MIN_MATRIX_SIZE, max=MAX_MATRIX_SIZE))
        os._exit(1)
    return size

def matrixGenerator(size):
    matrixResult=[]
    for i in range(0, size):
        matrixRow=[]
        for j in range(0, size):
            matrixRow.append(random.randint(MIN_MATRIX_CELL_VALUE, MAX_MATRIX_CELL_VALUE))
        matrixResult.append(matrixRow)
    return matrixResult

def printMatrix(alias,matrix):
    print("\nMatrix: {alias}".format(alias=alias))
    for row in matrix:
        for col in row:
            print("\t{col}\t".format(col=col),end="")
        print("\n",end="")

def secuentialMult(size, matrix_a, matrix_b):
    matrixResult = []
    for i in range(0, size):
        rowResult = [None] * size
        for j in range(0,size):
            for k in range(0,size):
                rowResult[j] = matrix_a[i][k] * matrix_b[k][j]
        matrixResult.append(rowResult)
    return matrixResult

def multiplyRow(size, matrix_a, matrix_b, rowResult, rowIndex):
    for j in range(0,size):
        for k in range(0,size):
            rowResult[j]= matrix_a[rowIndex][k]*matrix_b[k][j]

def concurrentMult(size, matrix_a, matrix_b):
    matrixResult = []
    threads = []

    for i in range(0, size):
        rowResult = [None] * size
        thread = threading.Thread(target=multiplyRow, args=(size, matrix_a, matrix_b, rowResult, i))
        thread.start()
        threads.append(thread)
        matrixResult.append(rowResult)

    for thread in threads:
        thread.join()

    return matrixResult

def compareMatrix(size, matrix_a, matrix_b):
    for i in range(0, size):
        for j in range(0, size):
            if matrix_a[i][j] != matrix_b[i][j]:
                return False
    return True

def main():
    size = validateArgs(sys.argv)
    matrix_a = matrixGenerator(size)
    matrix_b = matrixGenerator(size)
    matrix_secuential = secuentialMult(size,matrix_a,matrix_b)
    matrix_concurrent = concurrentMult(size,matrix_a,matrix_b)

    printMatrix("A", matrix_a)
    printMatrix("B", matrix_b)
    printMatrix("Secuencial", matrix_secuential)
    printMatrix("Concurrente", matrix_concurrent)

    resultCompare = "iguales" if compareMatrix(size, matrix_secuential, matrix_concurrent) else "distintas"
    print("Las matrices concurrente y secuencial son {compare}".format(compare=resultCompare))

main()

Overwriting tp2.py


In [ ]:
!python3 tp2.py 1

El tamaño debe estar entre 5 y 20


In [ ]:
!python3 tp2.py 5


Matrix: A
	26		4		-6		-24		7	
	16		18		-2		18		31	
	-13		30		-21		-14		25	
	-1		-11		0		-22		-21	
	22		16		31		-3		-21	

Matrix: B
	-1		-1		25		-26		26	
	26		-19		32		-8		19	
	15		11		-16		9		-24	
	7		5		-23		18		-17	
	17		27		17		-1		-16	

Matrix: Secuencial
	119		189		119		-7		-112	
	527		837		527		-31		-496	
	425		675		425		-25		-400	
	-357		-567		-357		21		336	
	-357		-567		-357		21		336	

Matrix: Concurrente
	119		189		119		-7		-112	
	527		837		527		-31		-496	
	425		675		425		-25		-400	
	-357		-567		-357		21		336	
	-357		-567		-357		21		336	
Las matrices concurrente y secuencial son iguales
